# Best-practices for Cloud-Optimized Geotiffs

**Part 2. Multiple COGs**

This notebook goes over ways to construct a multidimensional xarray DataArray from many 2D COGS

In [ ]:
import dask
import s3fs
import intake
import os
import xarray as xr
import pandas as pd

In [ ]:
# use the same GDAL environment settings as we did for the single COG case
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           AWS_NO_SIGN_REQUEST='YES',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000')
os.environ.update(env)

In [ ]:
# set up a connection with credentials and other settings
s3 = s3fs.S3FileSystem(anon=True)
objects = s3.ls('sentinel-s1-rtc-indigo/tiles/RTC/1/IW/10/T/ET/2020/')
images = ['s3://' + obj + '/Gamma0_VV.tif' for obj in objects]
print(len(images))
images[:6] #january 2020 scenes

## GDAL VRT

A GDAL VRT file is an XML format that can group together many separate files into separate bands. It's common to create such a file with a the GDAL command line tool `gdalbuildvrt`, illustrated below:

In [ ]:
#step 1) write a file list that points to the data. GDAL requires special prefixes for this /vsis3/ or /vsicurl/
with open('files.txt', 'w') as f:
    lines = [x.replace('s3://', '/vsis3/') + '\n' for x in images[:6]]
    f.writelines(lines)

In [ ]:
%%time
# step 2) create a VRT file
!gdalbuildvrt stack.vrt -separate -input_file_list files.txt 

In [ ]:
%%time
# step 4) open with xarray
chunks=dict(band=1, x=2745, y=2745)
da = xr.open_rasterio('stack.vrt', chunks=chunks)
da

In [ ]:
# step 5) optionally modify coordinates (e.g. time dimension extracted from file name)
da = da.rename({'band':'time'})
da['time'] = [pd.to_datetime(x[60:68]) for x in images[:6]]

#### Recap

1. `xr.open_rasterio(stack.vrt)` stores band coordinates as sequential integers (we lose file name and metadata from each individual COG, so it's common to alter the coordinates after opening the dataset)
2. data is tied to a reference to a local file ('stack.vrt'), which can cause problems with distributed computing if you don't have access to the local filesystem

## intake-xarray

[intake-xarray](https://github.com/intake/intake-xarray) is a plugin for the intake library. It uses fsspec/s3fs under the hood to facilitate loading data into python objects. the function `intake.open_rasterio()` accepts a list of paths. it returns an intake object with a `to_dask()` function that returns an xarray DataArray

In [ ]:
%%time
# ~ 1s for 6 files

# this loads the image ID into xarray's band coordinates. 

pattern = 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/10/T/ET/2020/{band}/Gamma0_VV.tif'
chunks=dict(band=1, x=2745, y=2745)
sources = intake.open_rasterio(images[:6], chunks=chunks, path_as_pattern=pattern, concat_dim='band')
da = sources.to_dask() 
da

#### recap:

* This is a convient way to avoid constructing a VRT and load a bunch of COGs. It works well as long as the COG urls follow a distinct pattern. Metadata is also lost (we have attributes from the first COG, not others)

## Custom

You can also just use xarray and dask to construct a larger datacube from many COGS. 

In [ ]:
%%time

# 4 - 8 s
# Load all the images

chunks=dict(band=1, x=2745, y=2745)
dataArrays = [xr.open_rasterio(url, chunks=chunks) for url in images]

# note use of join='override' b/c we know these COGS have the same coordinates
da = xr.concat(dataArrays, dim='band', join='override', combine_attrs='drop')
da = da.rename({'band':'time'})
da['time'] = [pd.to_datetime(x[60:68]) for x in images]
da

#### recap:

* The cell above is essential a for-loop that iterates over each COG in sequence. 50ms-200ms * 80 ~ 4-16 seconds. The next notebook will look at using Dask to speed things up by opening the files in parallel.

## Visualize

Here is an example of interactive visualization again using hvplot. Since we're using full resolution arrays it's key to set the `rasterize=True` keyword argument. That uses the datashader library to pre-render images before sending them to the browser.

This is extremely powerful because, resolution updates as you soom in, and you can scrub through the data cube with an interactive slider widget

In [ ]:
import hvplot.xarray
da.hvplot.image(rasterize=True, aspect='equal', cmap='gray', clim=(0,0.4))